In [2]:
import glob
import os
import pandas as pd

In [5]:
TILE_DIR = '/Users/darylwilding-mcbride/Downloads/190719_Hela_Ecoli_1to1_01-tiles/tile-33/pre-assigned'
BASE_TILE_URL = 'https://dwm-tims.s3-ap-southeast-2.amazonaws.com/190719_Hela_Ecoli_1to1_01-tiles/tile-33'

In [6]:
TILE_EDGE_LENGTH_PIXELS = 910

In [16]:
file_labels = []
for file in glob.glob("{}/*.txt".format(TILE_DIR))[:2]:
    tile_name = os.path.basename(file).split('.')[0]
    labels_df = pd.read_csv(file, sep=' ', names=['class_idx','x','y','width','height'])
    classes_d = {}
    for group_name,group_df in labels_df.groupby(['class_idx'], as_index=False):
        charge_state = int(group_name)+1
        geometry_l = []  # list of instances of this class
        for group_idx in range(len(group_df)):
            instance_label_df = group_df.iloc[group_idx]
            # x,y coordinates is the centre of the rectangle
            # origin for YOLO is top-left
            # origin for Labelbox is top-left
            centre_x_pixels = int(instance_label_df.x * TILE_EDGE_LENGTH_PIXELS)
            centre_y_pixels = int(instance_label_df.y * TILE_EDGE_LENGTH_PIXELS)
            width_pixels = int(instance_label_df.width * TILE_EDGE_LENGTH_PIXELS)
            height_pixels = int(instance_label_df.height * TILE_EDGE_LENGTH_PIXELS)

            x_lower_pixels = centre_x_pixels - int(width_pixels / 2)
            y_lower_pixels = centre_y_pixels - int(height_pixels / 2)
            x_upper_pixels = centre_x_pixels + int(width_pixels / 2)
            y_upper_pixels = centre_y_pixels + int(height_pixels / 2)

            vertices = []
            vertices.append({"x":x_lower_pixels,"y":y_lower_pixels})
            vertices.append({"x":x_upper_pixels,"y":y_lower_pixels})
            vertices.append({"x":x_upper_pixels,"y":y_upper_pixels})
            vertices.append({"x":x_lower_pixels,"y":y_upper_pixels})

            geometry_d = {"geometry":vertices}
            geometry_l.append(geometry_d)
        # classes_d is a dictionary of the classes, each dictionary entry is a list of geometries for each class instance
        classes_d["charge-{}".format(charge_state)] = geometry_l
        # consolidate the labels for this file
        file_label_d = {}
        file_label_d["prediction_label"] = classes_d
        file_label_d["image_url"] = "{}/{}.png".format(BASE_TILE_URL, tile_name)
        file_label_d["external_id"] = tile_name
    file_labels.append(file_label_d)

In [17]:
file_labels

[{'prediction_label': {'charge-1': [{'geometry': [{'x': 5, 'y': 599},
      {'x': 119, 'y': 599},
      {'x': 119, 'y': 677},
      {'x': 5, 'y': 677}]},
    {'geometry': [{'x': 780, 'y': 569},
      {'x': 894, 'y': 569},
      {'x': 894, 'y': 669},
      {'x': 780, 'y': 669}]},
    {'geometry': [{'x': 215, 'y': 382},
      {'x': 329, 'y': 382},
      {'x': 329, 'y': 988},
      {'x': 215, 'y': 988}]},
    {'geometry': [{'x': -20, 'y': 624},
      {'x': 94, 'y': 624},
      {'x': 94, 'y': 702},
      {'x': -20, 'y': 702}]},
    {'geometry': [{'x': 655, 'y': 624},
      {'x': 769, 'y': 624},
      {'x': 769, 'y': 700},
      {'x': 655, 'y': 700}]},
    {'geometry': [{'x': 105, 'y': 675},
      {'x': 219, 'y': 675},
      {'x': 219, 'y': 753},
      {'x': 105, 'y': 753}]},
    {'geometry': [{'x': 595, 'y': 587},
      {'x': 709, 'y': 587},
      {'x': 709, 'y': 665},
      {'x': 595, 'y': 665}]}],
   'charge-2': [{'geometry': [{'x': 231, 'y': 579},
      {'x': 319, 'y': 579},
      {'x':